In [1]:
import torch
import tensorflow as tf

from keras.backend.tensorflow_backend import set_session
##********************************************************
## For GPU
#
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.50
set_session(tf.Session(config=config))

#
##********************************************************

from keras import backend as K


from SSD_generate_anchors import generate_ssd_priors

from CustomDataLoaderTorch import DataAugmentor as DA1
from CustomDataLoaderTF import DataAugmentor as DA2

from utility import match as match_np
from utility import decode as decode_np
from utility import non_maximum_supression

from box_utils import match as match_th
from box_utils import decode as decode_th
from box_utils import nms

from pathlib import Path
import collections
from ssd300_model import SSD300
from skimage.io import imread, imshow
import cv2
import numpy as np

from Detector import Detect  as Detect_np
from detection import Detect as Detect_th

Using TensorFlow backend.


In [2]:
root                 = Path.home()/'Documents'/'DATASETS'/'VOCdevkit'


In [3]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

priors = generate_ssd_priors(specs).astype(np.float32)

In [4]:
batch_size = 4

loader_np  = DA2()
data_np    = loader_np.flow_from_directory(root         = root,
                                       data_folder  = ['VOC2007', 'VOC2012'],
                                       target_size  = (300,300),
                                       batch_size   = batch_size,
                                       shuffle      = False,
                                       priors       = priors
                                        )

In [5]:
sample_np = data_np[0]
images_np, targets_np = sample_np

loc_data  = targets_np[:,:,:4]
conf_data = targets_np[:,:,4:]



In [6]:
test_np = Detect_np(num_classes=21, bkg_label  = 0, top_k=200, conf_thresh=0.01, nms_thresh=0.45, variances=[0.1, 0.2])

In [7]:
detector_np = test_np.forward(loc_data = loc_data, conf_data=conf_data, priors = priors)

In [8]:
print(detector_np.shape)
print(detector_np.shape)

(4, 21, 200, 5)
(4, 21, 200, 5)


In [9]:
loc_data_th  = torch.from_numpy(loc_data).float()
conf_data_th = torch.from_numpy(conf_data).float()
priors_th    = torch.from_numpy(priors).float()

In [10]:
test_th = Detect_th(num_classes=21, bkg_label=0, top_k=200, conf_thresh=0.01, nms_thresh=0.45, variance=[0.1, 0.2])

In [11]:
detector_th = test_th.forward(loc_data=loc_data_th, conf_data=conf_data_th, prior_data=priors_th)

In [12]:
print(detector_th.size())
# print(detector_th[1])

torch.Size([4, 21, 200, 5])


In [13]:
a = detector_np == detector_th.numpy()
np.sum(a)

83845

In [14]:
a

array([[[[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         ...,
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True]],

        ...,

        [[ True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True],
         [ True,  